In [153]:
import pandas as pd

In [180]:
tweets_df = pd.read_csv('data/tweets.csv')
tweets_df.head()
tweets_df.shape

(14640, 15)

In [181]:
import re

stops = {"needn't", 'we', 'our', 'my', 'any', 'few', "it's", "that'll", 'do', 'hers', 'because', 
         'yourself', "haven't", 'will', 'with', "ain't", "mustn't", 'these', 'where', 'so', 'all', 
         'your', 'over', 'themselves', "should've", 'about', 'ours', 'they', 'after', "weren't", 
         'by', 'was', 'being', 'before', 'both', 'myself', 'has', 'into', 'as', "couldn't", "we're",
         'ourselves', 'there', 'under', "you'd", 'than', 'him', 'them', 'yourselves', 'and', 'no', 
         'same', 'whom', 'through', 'again', 'haven', 'an', 'some', 'nor', 'she', '&amp;', "i'll",
         'while', 'at', 'further', 'out', "wasn't", 'himself', 'which', "didn't", 'am', "shouldn't", 
         'theirs', 'are', 'his', 'doing', 'it', 'that', 'me', "weren't", 'does', "hasn't", 
         'most', 'down', "shouldn't", 'more', "hadn't", 'why', "aren't", 'have', 'then', "mustn't", 
         'during', 're', "didn't", 'when', 'to', 'on', "don't", "shan't", 'did', 'its', 'a', 'in', 
         'own', 'off', 'itself', "wouldn't", 'such', 'can', 'each', 'yours', 'is', 'against', 
         'between', 'for', 'what', "she's", 'i', "you'll", 'here', "couldn't", 'above', 'from', 
         'if', 'you', 'other', 'who', 'but', "mightn't", 'those', 'he', 'not', 'once', 'how', 'too', 
         'this', 'were', 'very', 'been', "you're", 'be', 'or', 'now', "you've", 'the', 'until', 
         'having', 'only', 'their', 'of', 'should', 'had', "isn't", 'below', "doesn't", "won't", 
         'herself', 'just', 'her', "i've", "i'm"}


print(len(stops))

157


In [182]:
import emoji
from nltk.stem.porter import PorterStemmer

def convert_emoji_to_text(emoji_text):
    text_with_aliases = emoji.demojize(emoji_text)
    return text_with_aliases

def clean_tweet(tweet:str) -> str:

    converted_text = convert_emoji_to_text(tweet)
    
    # remove links and make lowercase
    clean_tweet = re.sub(r'(www|http)\S*', '', converted_text).lower()
    clean_tweet = re.sub(r'_|:|-', ' ',clean_tweet)

    # replace ’ char with ' char
    clean_tweet = re.sub(r'’', "'", clean_tweet)

    # if words are present in stop words set then remove
    word_tokens = clean_tweet.split(' ')
    filtered_words = [word for word in word_tokens if not word in stops]

    clean_tweet = " ".join(filtered_words)
    
    # remove twitt handles, non-unicode chars, elipsis, & front slash
    clean_tweet = re.sub(r'(@[\S]*|[^\x00-\x7F]+|\.+|\/)', ' ', clean_tweet)
  
    # remove remaining punctuation and any non letter characters
    clean_tweet = re.sub(r'[^a-z\s]+', '', clean_tweet)
    
    # stemming words
    stemmer = PorterStemmer()
    clean_tweet = [stemmer.stem(word) for word in clean_tweet]
    clean_tweet = ''.join(clean_tweet)

    # Replace multiple spaces with a single space
    clean_tweet = re.sub(r'\s+', ' ', clean_tweet)
    
    return clean_tweet

In [183]:
# clean tweets 
tweets_df['clean_tweets'] = tweets_df['text'].apply(clean_tweet)
# tweets_df['clean_tweets'] = tweets_df.iloc[:,5].apply(clean_tweet2)

In [184]:
import random 

random_indices = random.sample(range(len(tweets_df)), 10)

# check random tweets before/after cleaning
for i in random_indices:
    print(tweets_df['text'][i])
    print(tweets_df['clean_tweets'][i])
    print('\n')

twitt = "November 19 is #NationalDayofMonaco. Did you know that @AeroportNice serves the Principality of Monaco? Beginning 5/6/24, you can discover Monaco with https://t.co/s0jk5HxcQZ @americanair's nonstop flights from #PHLAirport to Nice. "
clean_twitt = clean_tweet2(twitt)
print(twitt)
print(clean_twitt)

@AmericanAir hi my flight to DFW 2463 was Cancelled Flightled tonight. Called in and left my number for call back it's been 3hrs please help.
 hi flight dfw cancelled flightled tonight called left number call back hrs please help 


@JetBlue true. Maybe. Wish I had expedited security haha
 true maybe wish expedited security haha


@AmericanAir would it be possible to change my flight on your website? Many thanks
 would possible change flight website many thanks


@SouthwestAir why are flights still getting out of Logan now? You couldn't manage to get one out at 6 AM, when it wasn't snowing!
 flights still getting logan now manage get one am snowing


@VirginAmerica was wondering if you guys recieved my dm and we're able to potentially respond asap
 wondering guys recieved dm able potentially respond asap


@SouthwestAir has the winner for #DestinationDragons been announced yet? If not when will they be??
 winner destinationdragons announced yet be


@united I can’t go back to the airpo

In [185]:
# save dataframe to new csv file
tweets_df.to_csv('data/clean_tweets_v2.csv', index=False) 